<a href="https://colab.research.google.com/github/asigalov61/SuperPiano/blob/master/Super_Piano_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Super Piano 2

MAKE YOUR OWN SOTA PIANO MUSIC AI MODEL IN UNDER 4 HOURS !!! :)

Based on Yuankui Lee's repo and code https://github.com/djosix/Performance-RNN-PyTorch

MAESTRO Dataset is courtesy of Google Magenta Team and it is distributed under Attribution-NonCommercial-ShareAlike 4.0 International license.

So keep this in mind and respect everyone's copyright, please :)

Huge thanks go out to all people who shared these amazing code contributions and made this Colab notebook possible :) Thank you so much, guys!

## Setup the environment

In [ ]:
#@title Install all dependencies and requrements
print('3..2..1..lets do it')
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install numpy
%tensorflow_version 1.x
!pip install tensorflow-gpu==1.15
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  
import tensorflow as tf
!v1.disable_v2_behavior()
!pip install tensorboardX
!pip install tensorboard
!pip install progress
!pip install pretty-midi
!pip install pypianoroll
!pip install matplotlib
!pip install mir_eval
!pip install librosa
!pip install pyFluidSynth
!apt install fluidsynth #Pip does not work for some reason. Only apt works
!pip install midi2audio
!git clone https://github.com/asigalov61/Performance-RNN-PyTorch
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 /content/font.sf2
from midi2audio import FluidSynth
from google.colab import output
from IPython.display import display, Javascript, HTML, Audio
!nvidia-smi
print('Success :) Everything is installed and should work fine :) Enjoy!')

### Download and Unzip training MIDIs DataSet

In [ ]:
#@title (The Best Choice/Works best stand-alone) Super Piano 2 Original 2500 MIDIs of Piano Music
%cd /content/Performance-RNN-PyTorch/dataset/midi
!wget 'https://github.com/asigalov61/SuperPiano/raw/master/Super_Piano_2_MIDI_DataSet_CC_BY_NC_SA.zip'
!unzip -j 'Super_Piano_2_MIDI_DataSet_CC_BY_NC_SA.zip'

In [ ]:
#@title (Second Best Choice/Works best stand-alone) Alex Piano Only Drafts Original 1500 MIDIs 
%cd /content/Performance-RNN-PyTorch/dataset/midi
!wget 'https://github.com/asigalov61/AlexMIDIDataSet/raw/master/AlexMIDIDataSet-CC-BY-NC-SA-All-Drafts-Piano-Only.zip'
!unzip -j 'AlexMIDIDataSet-CC-BY-NC-SA-All-Drafts-Piano-Only.zip'

In [ ]:
#@title (Optional) Google Magenta MAESTRO Piano MIDI Dataset (~1300 MIDIs)
%cd /content/Performance-RNN-PyTorch/dataset/midi
!wget 'https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip'
!unzip -j maestro-v2.0.0-midi.zip

### Prepare and pre-process your MIDI DataSet for training

In [ ]:
#@title This may take a while, especially on the large DataSets, so please be patient
number_of_parallel_threads = 64 #@param {type:"slider", min:1, max:64, step:1}
%cd /content/Performance-RNN-PyTorch
!python3 preprocess.py '/content/Performance-RNN-PyTorch/dataset/midi' '/content/Performance-RNN-PyTorch/dataset/processed' $number_of_parallel_threads

### (Optional) Activate Tensorboard to monitor the progress of the model during training. You can also activate this cell at any other time to view logs/records of all training runs

In [ ]:
#@title Tensorboard Graphs and Stats
# Load the TensorBoard notebook extension
%reload_ext tensorboard
import tensorflow as tf
import datetime, os
%tensorboard --logdir /content/Performance-RNN-PyTorch/runs

###Train your model quickly here :) 

WARNING: Created/resulting Model may produce (partially) plagiarized (overfitted) output. Excercise care and respect the copyright, please :) NOTE: You can manipulate provided variables below to further influence/improve generated output. Only the first batch is downloaded and plotted.

In [ ]:
#@title Main Training Loop
save_model_every = 200 #@param {type:"slider", min:0, max:1000, step:10}
number_of_batches = 256 #@param {type:"slider", min:0, max:512, step:1}
window_size = 256 #@param {type:"slider", min:0, max:512, step:1}
control_ratio = 0.8 #@param {type:"number"}
teacher_forcing_ratio = 0.8 #@param {type:"number"}
stride_size = 0
%cd /content/Performance-RNN-PyTorch
!python3 train.py -s save/myModel.sess -d '/content/Performance-RNN-PyTorch/dataset/processed' -i $save_model_every -b $number_of_batches -w $window_size -c $control_ratio -T $teacher_forcing_ratio -L

iter 890, loss: 2.4750800132751465
iter 891, loss: 2.397463798522949
iter 892, loss: 2.4991939067840576
iter 893, loss: 2.4755849838256836
iter 894, loss: 2.4851131439208984
iter 895, loss: 2.491077423095703
iter 896, loss: 2.5439555644989014
iter 897, loss: 2.57888126373291
iter 898, loss: 2.436415433883667
iter 899, loss: 2.557011127471924
iter 900, loss: 2.494699239730835
iter 901, loss: 2.503755807876587
iter 902, loss: 2.393918514251709
iter 903, loss: 2.5388379096984863
iter 904, loss: 2.4078891277313232
iter 905, loss: 2.403362989425659
iter 906, loss: 2.6467843055725098
iter 907, loss: 2.7207515239715576
iter 908, loss: 2.361618757247925
iter 909, loss: 2.6313092708587646
iter 910, loss: 2.523188352584839
iter 911, loss: 2.5838375091552734
iter 912, loss: 2.465594530105591
iter 913, loss: 2.4437084197998047
iter 914, loss: 2.4919092655181885
iter 915, loss: 2.506157875061035


###Generate, Plot, Graph, Save, Download, and Render the resulting output

NOTES: Control takes commands or a processed .data file path.
E.g., "PITCH_HISTOGRAM;NOTE_DENSITY" like "2,0,1,1,0,1,0,1,1,0,0,1;4", or ";3" which gives all pitches the same probability or a /path/to/processed/midi/file.data" to use the specific control sequence from the given processed data file. Option -S stands for Stochastic Beam Search (use True or False after the option) (-c takes C,C#,D,D#,E,F,F#,G,G#,A,A#,B;note_density -S stands for Stochaistic Beam Search -g is a Greedy Ratio)

In [ ]:
#@title
number_of_tokens_to_generate = 1024 #@param {type:"slider", min:128, max:8192, step:64}
model_temperature = 0.8 #@param {type:"number"}
number_of_batches_and_files_to_generate = 1 #@param {type:"slider", min:0, max:16, step:1}
full_model_path_and_file_name = "/content/Performance-RNN-PyTorch/save/myModel.sess" #@param {type:"string"}
extra_flags = "-S" #@param {type:"string"}
generation_control_seed = "" #@param {type:"string"}
greedy_ratio = 0.8 #@param {type:"number"}

%cd /content/Performance-RNN-PyTorch

!python3 generate.py -l $number_of_tokens_to_generate -T $model_temperature -b $number_of_batches_and_files_to_generate -s $full_model_path_and_file_name $extra_flags $greedy_ratio $generation_control_seed

print('Successfully exported the output to output-000.mid')

from google.colab import files

files.download('/content/Performance-RNN-PyTorch/output/output-000.mid')
FluidSynth("/content/font.sf2").midi_to_audio('/content/Performance-RNN-PyTorch/output/output-000.mid','output_wav.wav')

# set the src and play
Audio("output_wav.wav")

In [ ]:
#@title Plot and Graph the Output :) Only first batch MIDI file is plotted and displayed 
graphs_length_inches = 18 #@param {type:"slider", min:0, max:20, step:1}
notes_graph_height = 6 #@param {type:"slider", min:0, max:20, step:1}
rendered_wav_graph_height = 3 #@param {type:"slider", min:0, max:20, step:1}
import librosa
import numpy as np
import pretty_midi
import pypianoroll
from pypianoroll import Multitrack, Track
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('SVG')
# For plotting
import mir_eval.display
import librosa.display
%matplotlib inline


midi_data = pretty_midi.PrettyMIDI('/content/Performance-RNN-PyTorch/output/output-000.mid')

def plot_piano_roll(pm, start_pitch, end_pitch, fs=100):
    # Use librosa's specshow function for displaying the piano roll
    librosa.display.specshow(pm.get_piano_roll(fs)[start_pitch:end_pitch],
                             hop_length=1, sr=fs, x_axis='time', y_axis='cqt_note',
                             fmin=pretty_midi.note_number_to_hz(start_pitch))



roll = np.zeros([int(graphs_length_inches), 128])
# Plot the output

track = Multitrack('/content/Performance-RNN-PyTorch/output/output-000.mid', name='track')
plt.figure(figsize=[graphs_length_inches, notes_graph_height])
fig, ax = track.plot()
fig.set_size_inches(graphs_length_inches, notes_graph_height)
plt.figure(figsize=[graphs_length_inches, notes_graph_height])
ax2 = plot_piano_roll(midi_data, 24, 84)
plt.show(block=False)

# Generate rendering (WAV)


#audio = midi_data.Synthesize()

#print(audio.shape)

#plt.figure(figsize=[graphs_length_inches, rendered_wav_graph_height])
#plt.plot(audio)
#plt.show(block=False)

#import IPython.display as ipd
#ipd.Audio(audio, rate=16000)